In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.svm import SVC

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import MinMaxScaler

In [2]:
churn_df = pd.read_csv("../data/task_data_training.csv")


In [3]:
churn_df.head()

,value_number_of_active_months,revenue,value_days_to_purchase,action_create_project,value_transactions_number,ws_users_activated,action_export_report,action_create_invoice,value_regular_seats,action_project_budget,action_time_entries_via_tracker,action_screenshots,churned_status
0,0,184.925,2,5,6,3,8,0,3,0,0,0.0,No
1,3,395.122,98,3,12,2,3,0,3,9,0,1.0,No
2,1,25.974,0,0,2,1,0,0,1,0,0,1.0,Yes
3,1,406.068,53,3,12,3,0,0,3,0,0,1.0,No
4,2,25.974,1,5,2,1,8,6,1,0,0,0.0,No


In [4]:
X = churn_df.drop(['churned_status'], axis=1)
y = churn_df['churned_status']

In [5]:
# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Fit the scaler to your data and transform it
normalized_data = scaler.fit_transform(X)